## Efficient Yelp API Calls (Core)
For this assignment, you will be working with the Yelp API.

As before, you will use the Yelp API to search your favorite city for a cuisine type of your choice.

Extract all of the results from your search and compile them into one dataframe using a for loop as shown in the lesson "Code for Efficient API Extraction"

Save your notebook, commit the change to your repository and submit the repository URL for this assignment.

In [1]:
# Standard Imports
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
# Additional Imports
import os, json, math, time
from yelpapi import YelpAPI
from tqdm.notebook import tqdm_notebook

### Load Credentials and Create Yelp API Object

In [2]:
# Load API Credentials
with open(r"C:\Users\bandi\.secret\yelp_api.json") as f:
    login = json.load(f)
login.keys()

dict_keys(['client-id', 'api-key'])

In [3]:
# Instantiate YelpAPI Variable
yelp_api = YelpAPI(login['api-key'], timeout_s = 5.0)
yelp_api

### Define Search and File Paths

In [16]:
# set our API call parameters 
LOCATION = 'Baltimore, MD'
TERM = 'Ramen'

In [17]:
# Name the file to save results
JSON_FILE = "Data/results_MD_ramen.json"
JSON_FILE

'Data/results_MD_ramen.json'

### Check if Json File exists and Create it if it doesn't

In [18]:
## Check if JSON_FILE exists
file_exists = os.path.isfile(JSON_FILE)

## If it does not exist: 
if file_exists == False:
    
    ## CREATE ANY NEEDED FOLDERS
    # Get the Folder Name only
    folder = os.path.dirname(JSON_FILE)
    
    ## If JSON_FILE included a folder:
    if len(folder)>0:
        # create the folder
        os.makedirs(folder,exist_ok=True)
        
        
    ## INFORM USER AND SAVE EMPTY LIST
    print(f'[i] {JSON_FILE} not found. Saving empty list to file.')
    
    
    # save an empty list
    with open(JSON_FILE,'w') as f:
        json.dump([],f) 
        
# If it exists, inform user
else:
    print(f"[i] {JSON_FILE} already exists.")

[i] Data/results_MD_ramen.json not found. Saving empty list to file.


### Load JSON FIle and account for previous results

In [19]:
## Load previous results and use len of results for offset
with open(JSON_FILE,'r') as f:
    previous_results = json.load(f)
    
## set offset based on previous results
n_results = len(previous_results)
print(f'- {n_results} previous results found.')

- 0 previous results found.


### Figure out how many pages of results we will need

In [20]:
# use our yelp_api variable's search_query method to perform our API call
results = yelp_api.search_query(location=LOCATION,
                                term=TERM,
                               offset=n_results)
results.keys()

dict_keys(['businesses', 'total', 'region'])

In [21]:
## How many results total?
total_results = results['total']
total_results

148

In [22]:
## How many did we get the details for?
results_per_page = len(results['businesses'])
results_per_page

20

In [23]:
# Use math.ceil to round up for the total number of pages of results.
n_pages = math.ceil((total_results - n_results) / results_per_page)
n_pages

8

### Add this page of results to .json file

In [24]:
# join new results with old list with extend and save to file
previous_results.extend(results['businesses'])  
with open(JSON_FILE,'w') as f:
     json.dump(previous_results,f,indent = 4)

### For Loop to call each page

In [25]:
for i in tqdm_notebook(range(1, n_pages + 1)):

    ## The block of code we want to TRY to run
    try:        
        # Introduce a short delay to respect API rate limits
        time.sleep(0.2)
        
        ## Read in results in progress file and check the length
        with open(JSON_FILE, 'r') as file:
            previous_results = json.load(file)
        
        ## Save number of results to use as offset
        n_results = len(previous_results)
        
        ## Use n_results as the OFFSET 
        results = yelp_api.search_query(location = LOCATION,
                                        term = TERM,
                                        offset = len(previous_results))

        ## Append new results and save to file
        previous_results.extend(results['businesses'])
        with open(JSON_FILE, 'w') as file:
            json.dump(previous_results, file)

            
## What to do if we get an error/exception.
    except Exception as e:
        # check if we are at rate limit
        if 'Too Many Requests for url' in str(e):
            print('Rate limit exceeded. Stop data collection.')
            break
        else:
            print(f'an error occured {e}')
            # optionally handle error differently
            continue

  0%|          | 0/8 [00:00<?, ?it/s]

In [26]:
# load final results
final_df = pd.read_json(JSON_FILE)
display(final_df.head(), final_df.tail())

,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
0,X19jig0ubkRVdHdOup5onQ,ramen-utsuke-baltimore-2,Ramen Utsuke,https://s3-media2.fl.yelpcdn.com/bphoto/BLOd7q...,False,https://www.yelp.com/biz/ramen-utsuke-baltimor...,463,"[{'alias': 'ramen', 'title': 'Ramen'}]",4.4,"{'latitude': 39.28356027844324, 'longitude': -...","[pickup, delivery]",$$,"{'address1': '414 Light St', 'address2': 'Ste ...",+14435632977,(443) 563-2977,2886.661832
1,JY72TrhFESLIJsXz5Na_iA,toki-underground-baltimore,Toki Underground,https://s3-media4.fl.yelpcdn.com/bphoto/8JPqZh...,False,https://www.yelp.com/biz/toki-underground-balt...,80,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",4.5,"{'latitude': 39.321619, 'longitude': -76.609174}","[delivery, restaurant_reservation, pickup]",$$,"{'address1': '2731 Greenmount Ave', 'address2'...",+14434495392,(443) 449-5392,1370.557673
2,wwfTFc9B_cKGE0sg-_533A,kippo-ramen-baltimore,Kippo Ramen,https://s3-media2.fl.yelpcdn.com/bphoto/RXcalJ...,False,https://www.yelp.com/biz/kippo-ramen-baltimore...,680,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",4.4,"{'latitude': 39.2841531081, 'longitude': -76.5...","[pickup, delivery]",$$,"{'address1': '606 S Bdwy', 'address2': '', 'ad...",+16672124841,(667) 212-4841,3094.473885
3,B82cpig_hNiBsM77tXMQmA,kajiken-baltimore,Kajiken,https://s3-media3.fl.yelpcdn.com/bphoto/DBalZZ...,False,https://www.yelp.com/biz/kajiken-baltimore?adj...,92,"[{'alias': 'ramen', 'title': 'Ramen'}]",4.5,"{'latitude': 39.327879, 'longitude': -76.616392}",[],NaN,"{'address1': '7 E 33rd St', 'address2': '', 'a...",+14438354617,(443) 835-4617,2167.559287
4,6Pgn8ON83YE-cTPoOMCm-A,dashi-ramen-baltimore,Dashi Ramen,https://s3-media1.fl.yelpcdn.com/bphoto/9nkZpt...,False,https://www.yelp.com/biz/dashi-ramen-baltimore...,8,"[{'alias': 'ramen', 'title': 'Ramen'}, {'alias...",5.0,"{'latitude': 39.2768302386569, 'longitude': -7...","[pickup, delivery]",NaN,"{'address1': '1065 S Charles St', 'address2': ...",+13016511349,(301) 651-1349,3627.044772


,id,alias,name,image_url,is_closed,url,review_count,categories,rating,coordinates,transactions,price,location,phone,display_phone,distance
143,Wq8jJ1X-L36OiV4seDkaYw,7-eleven-baltimore-26,7-Eleven,https://s3-media4.fl.yelpcdn.com/bphoto/nEnF7P...,False,https://www.yelp.com/biz/7-eleven-baltimore-26...,5,"[{'alias': 'convenience', 'title': 'Convenienc...",2.6,"{'latitude': 39.2938614405871, 'longitude': -7...","[delivery, pickup]",$,"{'address1': '1114 Ingleside Av', 'address2': ...",+14107475399,(410) 747-5399,10581.370033
144,pGPugxXaBn4_7e2eUY4VPA,7-eleven-catonsville-3,7-Eleven,https://s3-media1.fl.yelpcdn.com/bphoto/0YmU27...,False,https://www.yelp.com/biz/7-eleven-catonsville-...,4,"[{'alias': 'convenience', 'title': 'Convenienc...",2.8,"{'latitude': 39.2731536, 'longitude': -76.7526...","[delivery, pickup]",$,"{'address1': '1611 Edmondson Ave', 'address2':...",+14107884441,(410) 788-4441,12996.726394
145,plLY3F-oTVirLV8uerZOXQ,7-eleven-elkridge-3,7-Eleven,,False,https://www.yelp.com/biz/7-eleven-elkridge-3?a...,1,"[{'alias': 'convenience', 'title': 'Convenienc...",1.0,"{'latitude': 39.188869663407, 'longitude': -76...","[delivery, pickup]",$,"{'address1': '6590 Old Waterloo Rd', 'address2...",+14107995169,(410) 799-5169,20717.875905
146,f_5JsfzbjkQPWoIaI6hvOw,7-eleven-towson,7-Eleven,,False,https://www.yelp.com/biz/7-eleven-towson?adjus...,6,"[{'alias': 'convenience', 'title': 'Convenienc...",2.0,"{'latitude': 39.3992867255242, 'longitude': -7...",[delivery],$,"{'address1': '321 York Rd', 'address2': '', 'a...",+14108232461,(410) 823-2461,10021.418156
147,e4aPVutSt5Y5rZa-v2-TvA,7-eleven-hanover,7-Eleven,https://s3-media2.fl.yelpcdn.com/bphoto/qX8afF...,False,https://www.yelp.com/biz/7-eleven-hanover?adju...,3,"[{'alias': 'convenience', 'title': 'Convenienc...",3.0,"{'latitude': 39.1588731637158, 'longitude': -7...","[delivery, pickup]",$,"{'address1': '1401 Dorsey Rd', 'address2': '',...",+14108595730,(410) 859-5730,19025.149138


In [28]:
# check for duplicate ID's 
final_df.duplicated(subset='id').sum()

0

In [29]:
# save the final results to csv
final_df.to_csv('Data/final_results_MD_ramen.csv', index=False)